<a href="https://colab.research.google.com/github/pamslover/pamellaPfe/blob/hotfix/clustering_embedding.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [4]:
!pip install yake

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.2/60.2 kB 900.2 kB/s eta 0:00:00


In [5]:
import pandas as pd
import numpy as np
from sklearn.cluster import SpectralClustering
import itertools
from math import sqrt, floor
import os
import re
from os import path
from wordcloud import WordCloud
import matplotlib.pyplot as plt
import yake
from nltk.tokenize import RegexpTokenizer
from tqdm import tqdm

In [6]:
from transformers import BertTokenizer, BertModel
import torch
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE

## Yake
nous allons utiliser yake pour extraction des mots clés dans nos cv et job pour le clustering de notre corpus

In [7]:
def keywords_yake(sample_posts):
    # take keywords for each post & turn them into a text string "sentence"
    # create empty list to save our "sentences" to
    simple_kw_extractor = yake.KeywordExtractor()
    sentences = []
    for post in sample_posts:
        post_keyword = simple_kw_extractor.extract_keywords(post)
        sentence_output = ""
        for word, number in post_keyword:
            sentence_output += word + " "
        sentences.append(sentence_output)
    return sentences

    # use the sentences as input for brown clustering


def tokenizing_after_yake(sentences):
    # tokenize
    tokenizer = RegexpTokenizer(r'\w+')
    sample_data_tokenized = [w.lower() for w in sentences]
    sample_data_tokenized = [tokenizer.tokenize(i) for i in sample_data_tokenized]
    return sample_data_tokenized


class YakeHelperFunctions:

    def get_cluster_maybe(self, megacluster):
        cluster_list = [[]]
        list_index = 0
        for i in range(len(megacluster) - 1):
            if megacluster[i] < megacluster[i + 1]:
                cluster_list[list_index].append(megacluster[i])
            else:
                cluster_list[list_index].append(megacluster[i])
                list_index = list_index + 1
                cluster_list.append([])
        return cluster_list


# Clustering

## Spectralclustering

### clustering_embedding_word2vec

In [9]:
df = pd.read_csv('/content/drive/MyDrive/projet pfe/CvClean.csv')
jd = pd.read_csv("/content/drive/MyDrive/projet pfe/job.csv")

FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/projet pfe/CvClean.csv'

In [10]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
data = df['Resume'].astype(str)

In [ ]:
data.drop(0, inplace=True)
data.shape

In [ ]:
jd.shape

In [ ]:
vectors = pd.read_csv("/content/drive/MyDrive/projet pfe/word2vec/word2vec_cv_job.model", delim_whitespace=True, skiprows=[0], header=None)

In [ ]:
vectors

In [ ]:
#extraction des mots clés avec yake
keywords = keywords_yake(df['Resume'].astype(str))
keywords_tokenized = tokenizing_after_yake(keywords)
keywords_set = [ set(job) for job in keywords_tokenized]
#remove empty sets
keywords_set_no_empty = [x for x in keywords_set if x]

In [ ]:
def vectors_from_job(job):
        all_words = []
        for words in job:
                all_words.append(words)
        return vectors[vectors.index.isin(all_words)]

In [ ]:
# test function for getting job description embedding (in the dumbest way possible)
def doc_embed_from_job(job):
        test_vectors = vectors_from_job(job)
        if test_vectors.empty:
            return np.zeros(300) # Return a zero vector if no embeddings are found
        else:
            return test_vectors.mean()

In [ ]:
# get document embeddings for job
num_of_jobs = len(keywords_set_no_empty)
doc_embeddings = np.zeros([num_of_jobs,300])
# TODO: handle jobs where all word out vocabulary
for i in range(num_of_jobs):
        embeddings =np.array(doc_embed_from_job(keywords_set_no_empty[i]))
        if np.isnan(embeddings).any():
                doc_embeddings[i,:] = np.zeros([1,300])
        else:
                doc_embeddings[i,:] = embeddings

In [ ]:
doc_embeddings.shape

In [ ]:
number_of_clusters = floor(sqrt(num_of_jobs))

In [ ]:
clustering = SpectralClustering(assign_labels='discretize', random_state=42, n_neighbors=number_of_clusters).fit(doc_embeddings)

In [ ]:
pd.Series(clustering.labels_).value_counts()

In [ ]:
# explore our jobs by cluster number
jobs_subset = keywords_set_no_empty[0:num_of_jobs]
def get_keyword_set_by_cluster_number(number):
        cluster_index = list(clustering.labels_ == number)
        return list(itertools.compress(jobs_subset, cluster_index))

In [ ]:
get_keyword_set_by_cluster_number(3)

In [ ]:
def get_job_infos_by_cluster_number(number,  data, cluster):
        if len(cluster.labels_) != len(data):
            print(f"Length of labels: {len(cluster.labels_)}")
            print(f"Length of data: {len(data)}")
            raise ValueError("Number of labels does not match the number of data rows.")
        return data[cluster.labels_ == number]

def remove_unnecessary_word(text, liste_word):
        text = text.lower()
        for element in liste_word:
                text = re.sub(element,"",text)
        return text

In [ ]:
for i in range(num_of_jobs):
        print(f"cluster {i}:\n")
        print(get_job_infos_by_cluster_number(i, data, clustering))
        print("\n")

In [ ]:

for i in range(number_of_clusters):
        jobs_as_string = get_job_infos_by_cluster_number(i, data, clustering)\
        .to_string(index=True)
        jobs_as_string = remove_unnecessary_word(jobs_as_string,[])
        # Generate a word cloud image
        wordcloud = WordCloud().generate(jobs_as_string)

        # Display the generated image:
        # the matplotlib way:

        plt.imshow(wordcloud, interpolation='bilinear')
        plt.axis("off")
        plt.show()

refining cluster

In [ ]:
cluster_counts = pd.Series(clustering.labels_).value_counts()
max_cluster_size = number_of_clusters * 2
big_clusters = cluster_counts[cluster_counts > max_cluster_size]

In [ ]:
big_clusters

In [ ]:
# first
cluster_label = big_clusters.index[0]
sub_sample = data[clustering.labels_ == cluster_label]
sub_cluster_embeddings = doc_embeddings[clustering.labels_ == cluster_label]

In [ ]:
number_of_sub_clusters = floor(sqrt(sub_sample.shape[0]))

In [ ]:
sub_clustering = SpectralClustering(assign_labels='discretize', random_state=42, n_neighbors=number_of_sub_clusters, n_clusters=number_of_sub_clusters).fit(sub_cluster_embeddings)

In [ ]:
for i in range(number_of_sub_clusters):
        print(f"cluster {i}:\n")
        print(get_job_infos_by_cluster_number(i, sub_sample, sub_clustering))
        print("\n")

In [ ]:
pd.Series(sub_clustering.labels_).value_counts()

In [ ]:
get_job_infos_by_cluster_number(4, sub_sample, sub_clustering)

In [ ]:
jobs_from_cluster = get_job_infos_by_cluster_number(4, sub_sample, sub_clustering)
jobs_as_string = jobs_from_cluster.to_string(index=True)

In [ ]:
wordcloud = WordCloud().generate(jobs_as_string)
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis("off")

In [ ]:
for i in range(number_of_sub_clusters):
        jobs_as_string = get_job_infos_by_cluster_number(i, sub_sample, sub_clustering)\
        .to_string(index=True)
        # Generate a word cloud image
        wordcloud = WordCloud().generate(jobs_as_string)

        # Display the generated image:
        # the matplotlib way:

        plt.imshow(wordcloud, interpolation='bilinear')
        plt.axis("off")
        plt.show()

### clustering_embedding_bert

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/projet pfe/CvClean.csv')
jd = pd.read_csv("/content/drive/MyDrive/projet pfe/job.csv")
# Charger les modèles et les tokenizers fine-tunés
cv_model = BertModel.from_pretrained("/content/drive/MyDrive/projet pfe/BERT/model_bert_cv")
cv_tokenizer = BertTokenizer.from_pretrained("/content/drive/MyDrive/projet pfe/BERT/token_bert_cv")

job_model = BertModel.from_pretrained("/content/drive/MyDrive/projet pfe/BERT/model_bert_job")
job_tokenizer = BertTokenizer.from_pretrained("/content/drive/MyDrive/projet pfe/BERT/token_bert_job")


In [ ]:

def get_embeddings(texts, model, tokenizer, max_length=512):
    inputs = tokenizer(texts, return_tensors='pt', max_length=max_length, truncation=True, padding=True)
    with torch.no_grad():
        outputs = model(**inputs)
    return outputs.last_hidden_state.mean(dim=1)

In [ ]:
samples_cv = df['Resume'].astype(str)
samples_job = jd['job'].astype(str)

job_embeddings = get_embeddings(samples_job.tolist(), job_model, job_tokenizer)
cv_embeddings = get_embeddings(samples_cv.tolist(), cv_model, cv_tokenizer)

all_embeddings = torch.cat((cv_embeddings, job_embeddings), dim=0)

# Définir le nombre de clusters
num_clusters = floor(sqrt(len(all_embeddings)))

In [ ]:
clustering = SpectralClustering(assign_labels='discretize', random_state=42, n_clusters=num_clusters).fit(all_embeddings)

In [ ]:
# Visualisation des clusters avec t-SNE
tsne = TSNE(n_components=2, random_state=42)
reduced_embeddings = tsne.fit_transform(all_embeddings)

colors = ['red'] * len(samples_cv) + ['blue'] * len(samples_job)

plt.figure(figsize=(10, 7))
plt.scatter(reduced_embeddings[:, 0], reduced_embeddings[:, 1], c=colors, alpha=0.6)
plt.title("Clustering des Embeddings des CVs et Offres d'Emploi")
plt.xlabel("t-SNE Dimension 1")
plt.ylabel("t-SNE Dimension 2")
plt.show()

In [ ]:
# Afficher les clusters et générer des nuages de mots
def get_job_infos_by_cluster_number(number, data, cluster):
    return data[cluster.labels_ == number]

for i in range(num_clusters):
    print(f"Cluster {i}:\n")
    jobs_as_string = get_job_infos_by_cluster_number(i, samples_job, clustering).to_string(index=True)
    # Generate a word cloud image
    wordcloud = WordCloud().generate(jobs_as_string)
    plt.imshow(wordcloud, interpolation='bilinear')
    plt.axis("off")
    plt.show()

In [ ]:
import numpy as np
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler

# Supposons que vos embeddings sont stockés dans des tableaux NumPy
# Remplacez ces lignes par le chargement réel de vos données
bert_fine_tuning_cv = np.load('path_to_bert_fine_tuning_cv.npy')
bert_fine_tuning_job = np.load('path_to_bert_fine_tuning_job.npy')

# Combiner les vecteurs de CVs et d'offres d'emploi
all_vectors = np.concatenate([bert_fine_tuning_cv, bert_fine_tuning_job], axis=0)

# Optionnel : Normaliser les données pour une meilleure performance de K-means
scaler = StandardScaler()
all_vectors = scaler.fit_transform(all_vectors)

# Définir le nombre de clusters souhaité
n_clusters = 10  # Vous pouvez ajuster ce nombre selon vos besoins

# Appliquer K-means
kmeans = KMeans(n_clusters=n_clusters, random_state=42)
labels = kmeans.fit_predict(all_vectors)

# Séparer les labels des CVs et des offres d'emploi
cv_labels = labels[:len(bert_fine_tuning_cv)]
job_labels = labels[len(bert_fine_tuning_cv):]

# Afficher les résultats
print("Labels des CVs:", cv_labels)
print("Labels des offres d'emploi:", job_labels)


In [ ]:
# Calculer la similarité cosinus
similarity_matrix = cosine_similarity(all_vectors)

# Examiner les similarités au sein des clusters
def cluster_similarities(cluster_label, labels, similarity_matrix):
    indices = np.where(labels == cluster_label)[0]
    cluster_sim = similarity_matrix[np.ix_(indices, indices)]
    return cluster_sim

# Exemple d'examen pour le cluster 0
cluster_label = 0
cluster_sim = cluster_similarities(cluster_label, labels, similarity_matrix)

# Afficher les similarités au sein du cluster 0
print(f"Similarités au sein du cluster {cluster_label}:")
print(cluster_sim)



In [ ]:
# Visualiser les similarités avec une heatmap
plt.figure(figsize=(10, 8))
sns.heatmap(cluster_sim, annot=True, cmap='coolwarm', cbar=True)
plt.title(f"Heatmap des similarités au sein du cluster {cluster_label}")
plt.show()

In [ ]:
from sklearn.metrics import silhouette_score, silhouette_samples
import matplotlib.pyplot as plt

# Calculer le score de silhouette
silhouette_avg = silhouette_score(all_vectors, labels)
print(f"Le score de silhouette moyen est de : {silhouette_avg}")

# Calculer les échantillons de silhouette pour chaque point
sample_silhouette_values = silhouette_samples(all_vectors, labels)

# Calculer l'inertie
inertia = kmeans.inertia_
print(f"L'inertie du clustering est de : {inertia}")


In [ ]:
from sklearn.manifold import TSNE
import seaborn as sns

# Réduction de la dimensionnalité avec t-SNE
tsne = TSNE(n_components=2, perplexity=30, n_iter=300, random_state=42)
tsne_results = tsne.fit_transform(all_vectors)

# Visualiser les clusters
plt.figure(figsize=(10, 8))
sns.scatterplot(x=tsne_results[:, 0], y=tsne_results[:, 1], hue=labels, palette='viridis', legend='full')
plt.title('Visualisation des Clusters avec t-SNE')
plt.show()


In [ ]:
import umap

# Réduction de la dimensionnalité avec UMAP
umap_model = umap.UMAP(n_components=2, random_state=42)
umap_results = umap_model.fit_transform(all_vectors)

# Visualiser les clusters
plt.figure(figsize=(10, 8))
sns.scatterplot(x=umap_results[:, 0], y=umap_results[:, 1], hue=labels, palette='viridis', legend='full')
plt.title('Visualisation des Clusters avec UMAP')
plt.show()


In [ ]:
from sklearn.cluster import KMeans
import torch
import numpy as np
from transformers import BertModel, BertTokenizer, AutoModel

# Charger les embeddings des CVs et des offres d'emploi
# Load the model directly from the directory
cv_model = AutoModel.from_pretrained("/content/drive/MyDrive/projet pfe/bert_fine_tuned")
cv_embeddings = cv_model.embeddings.word_embeddings.weight.data.numpy()  # Extract embeddings

job_model = AutoModel.from_pretrained("/content/drive/MyDrive/projet pfe/bert_fine_tuned_job")
job_embeddings = job_model.embeddings.word_embeddings.weight.data.numpy()

# Combiner les embeddings pour clustering
all_embeddings = np.concatenate((cv_embeddings, job_embeddings), axis=0)

#all_embeddings = torch.cat((cv_embeddings, job_embeddings), dim=0).numpy()

# Appliquer K-means clustering
kmeans = KMeans(n_clusters=5, random_state=0).fit(all_embeddings)
labels = kmeans.labels_

# Séparer les labels pour les CVs et les offres d'emploi
cv_labels = labels[:len(cvs)]
job_labels = labels[len(cvs):]


In [ ]:
import joblib

# Sauvegarder le modèle K-means
joblib.dump(kmeans, 'kmeans_model.pkl')

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

# Calculer la similarité cosinus entre les embeddings des CVs et des offres d'emploi
similarity_matrix = cosine_similarity(cv_embeddings, job_embeddings)

# Exemple de visualisation des similarités
import pandas as pd

#similarity_df = pd.DataFrame(similarity_matrix, index=[f"CV {i+1}" for i in range(len(cvs))],
                             #columns=[f"Job {i+1}" for i in range(len(jobs))])
similarity_df = pd.DataFrame(similarity_matrix,
                             index=[f"CV {i+1}" for i in range(similarity_matrix.shape[0])],  # Use shape of similarity matrix
                             columns=[f"Job {i+1}" for i in range(similarity_matrix.shape[1])])  # Use shape of similarity matrix

print(similarity_df)


In [ ]:
# Trouver les meilleures correspondances en fonction des scores de similarité
def match_cvs_jobs(similarity_matrix):
    matches = []
    for i, similarities in enumerate(similarity_matrix):
        best_match_idx = similarities.argmax()
        matches.append((i, best_match_idx, similarities[best_match_idx]))
    return matches

matches = match_cvs_jobs(similarity_matrix)
print("Matching results (CV index, Job index, Similarity):", matches)


In [ ]:
from transformers import BertTokenizer, BertModel
import torch
from sklearn.cluster import KMeans
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd

# Charger le tokenizer et le modèle BERT
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

# Fonction pour obtenir les embeddings BERT
def get_bert_embeddings(texts):
    inputs = tokenizer(texts, return_tensors='pt', padding=True, truncation=True, max_length=512)
    with torch.no_grad():
        outputs = model(**inputs)
    embeddings = outputs.last_hidden_state.mean(dim=1)
    return embeddings

# Charger le modèle de clustering
kmeans = KMeans(n_clusters=5)
# Charger les clusters (vous devrez avoir sauvegardé les clusters)
kmeans.cluster_centers_ = pd.read_pickle('/content/kmeans_model.pkl')

# Fonction pour matcher les CVs aux offres d'emploi
def match_cvs_jobs(cv_texts, job_texts):
    cv_embeddings = get_bert_embeddings(cv_texts)
    job_embeddings = get_bert_embeddings(job_texts)
    similarity_matrix = cosine_similarity(cv_embeddings, job_embeddings)
    matches = []
    for i, similarities in enumerate(similarity_matrix):
        best_match_idx = similarities.argmax()
        matches.append((cv_texts[i], job_texts[best_match_idx], similarities[best_match_idx]))
    return matches


In [ ]:
!pip install streamlit


In [ ]:
import streamlit as st
import pandas as pd
from transformers import BertTokenizer, BertModel
import torch
from sklearn.metrics.pairwise import cosine_similarity

# Charger le tokenizer et le modèle BERT
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

# Fonction pour obtenir les embeddings BERT
def get_bert_embeddings(texts):
    inputs = tokenizer(texts, return_tensors='pt', padding=True, truncation=True, max_length=512)
    with torch.no_grad():
        outputs = model(**inputs)
    embeddings = outputs.last_hidden_state.mean(dim=1)
    return embeddings

# Fonction pour matcher les CVs aux offres d'emploi
def match_cvs_jobs(cv_texts, job_texts):
    cv_embeddings = get_bert_embeddings(cv_texts)
    job_embeddings = get_bert_embeddings(job_texts)
    similarity_matrix = cosine_similarity(cv_embeddings, job_embeddings)
    matches = []
    for i, similarities in enumerate(similarity_matrix):
        best_match_idx = similarities.argmax()
        matches.append((cv_texts[i], job_texts[best_match_idx], similarities[best_match_idx]))
    return matches

# Interface utilisateur Streamlit
st.title('Matching de CVs et Offres d\'Emploi')
st.write('Charger les fichiers CSV contenant les CVs et les offres d\'emploi.')

# Charger les fichiers CSV
cv_file = st.file_uploader('/content/drive/MyDrive/projet pfe/CvClean.csv', type='csv')
job_file = st.file_uploader('/content/drive/MyDrive/projet pfe/JobDescription.csv', type='csv')

if cv_file and job_file:
    cvs = pd.read_csv(cv_file)
    jobs = pd.read_csv(job_file)

    # Assurer que les colonnes de texte sont nommées 'text'
    cvs['Resume'] = cvs['Resume'].astype(str)
    jobs['jobs'] = jobs['jobs'].astype(str)

    # Matcher les CVs et les offres d'emploi
    matches = match_cvs_jobs(cvs['Resume'].tolist(), jobs['jobs'].tolist())

    st.write('Résultats du Matching:')
    for cv, job, similarity in matches:
        st.write(f'CV: {cv[:200]}... | Offre: {job[:200]}... | Similarité: {similarity:.2f}')

# Exécuter l'application avec la commande suivante dans le terminal:
# streamlit run app.py
